<a href="https://colab.research.google.com/github/shaktisd/-meta_llama_2finetuned_text_generation_summarization/blob/main/Gemma_Finetuning_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Using Google Gemma's Model

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 28.9 MB/s eta 0:00:00


In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

### Prerequisites
* nf4(4-bit NormalFloat(NF4)) : https://www.kaggle.com/code/lorentzyeung/what-s-4-bit-quantization-how-does-it-help-llama2


In [4]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in


In [7]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

- Albert Einstein

The


In [8]:
os.environ["WANDB_DISABLED"] = "false"

In [9]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [44]:
from datasets import load_dataset

#data_old = load_dataset("Abirate/english_quotes")
data = load_dataset("jppgks/twitter-financial-news-sentiment")
data = data.map(lambda samples: tokenizer(samples["input"]), batched=True)

Map:   0%|          | 0/9543 [00:00<?, ? examples/s]

Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [38]:
data_old

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [41]:
data

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 9543
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 2388
    })
})

In [ ]:
data['validation']['input'][100:200]

In [ ]:
data['validation']['output'][100:200]

In [46]:
def formatting_func(example):
    text = f"News: {example['input'][0]}\nSentiment: {example['output'][0]}"
    return [text]

In [47]:
data['train']

Dataset({
    features: ['input', 'output', 'instruction', 'input_ids', 'attention_mask'],
    num_rows: 9543
})

In [48]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/9543 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

In [57]:
text = "News: Stock market is at all time high"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

News: Stock market is at all time high
Sentiment: neutral
Sentiment: neutral
Sentiment: neutral
Sentiment: neutral
Sentiment: neutral


In [92]:
def get_sentiment(text):
    device = "cuda:0"
    instruction = "What is the sentiment of this News? Please choose an answer from negative/neutral/positive"
    input_text = f"News: {text}"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    r = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #print(r)
    result = r.split("\n")[1].replace("Sentiment: ","")
    return result

In [100]:
correct = 0
wrong = 0
index = 0
prediction_tag = ""
for input, output in zip(data['validation']['input'],data['validation']['output']) :
    result = get_sentiment(input)

    if output in result:
        prediction_tag = "CORRECT"
        correct += 1
    else:
        prediction_tag = "WRONG"
        wrong += 1
    print(f'PREDICTION {index}: {prediction_tag} , Input', input, 'Actual Output',output, 'Predicted Output ', result, )
    index += 1


#     text = f"News: {row}"
# device = "cuda:0"
# inputs = tokenizer(text, return_tensors="pt").to(device)

# outputs = model.generate(**inputs, max_new_tokens=20)

PREDICTION 0: CORRECT , Input $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5 Actual Output negative Predicted Output  negative sentiment
PREDICTION 1: CORRECT , Input $DELL $HPE - Dell, HPE targets trimmed on compute headwinds https://t.co/YRUHZw7cYl Actual Output negative Predicted Output  negative sentiment Sentimentr: negative
PREDICTION 2: CORRECT , Input $PRTY - Moody's turns negative on Party City https://t.co/MBD5TFGC4P Actual Output negative Predicted Output  negative
PREDICTION 3: WRONG , Input $SAN: Deutsche Bank cuts to Hold Actual Output negative Predicted Output  neutral
PREDICTION 4: WRONG , Input $SITC: Compass Point cuts to Sell Actual Output negative Predicted Output  neutral
PREDICTION 5: CORRECT , Input $TAP - Barclays cools on Molson Coors https://t.co/FJFsZWJHuH Actual Output negative Predicted Output  negative
PREDICTION 6: WRONG , Input $TAP: Barclays cuts to Equal Weight Actual Output negative Predicted Output  neutral
PREDICTION 7: CORRECT , Input

IndexError: list index out of range

In [99]:
text = data['validation']['input'][0]
get_sentiment(text)

'negative sentiment'

In [98]:
text

'$SAN: Deutsche Bank cuts to Hold'

In [97]:
data['validation']['output'][3]

'negative'

In [ ]:
text = "Quote: Outside of a dog, a book is man's"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Outside of a dog, a book is man's best friend.
Author: Nicolas Chamfort
Quote: The most wasted of all days is one


In [34]:
text = "Quote: Live as if you were to die"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Live as if you were to die tomorrow. Learn as if you were to live forever.
Author: Aung San Suu Kyi
